## Load the enhanced sets

In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from ast import literal_eval

# Load data from CSV files
data = pd.read_csv('Data/Cleaned_Enhanced_training.csv')
test_data = pd.read_csv('Data/Cleaned_Enhanced_test.csv')

# Convert string representations of lists back to actual lists
data['embeddings'] = data['embeddings'].apply(literal_eval)
test_data['embeddings'] = test_data['embeddings'].apply(literal_eval)

# Flatten the embeddings
num_embedding_features = len(data['embeddings'].iloc[0])
for i in range(num_embedding_features):
    data[f'emb_{i}'] = data['embeddings'].apply(lambda x: x[i])
    test_data[f'emb_{i}'] = test_data['embeddings'].apply(lambda x: x[i])

k_fold = data
# Drop the original embeddings column and other non-feature columns
data.drop(['embeddings', 'sentence', 'id'], axis=1, inplace=True)
test_data.drop(['embeddings', 'sentence', 'id'], axis=1, inplace=True)



/tmp/ipykernel_31304/2144286977.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'emb_{i}'] = data['embeddings'].apply(lambda x: x[i])
/tmp/ipykernel_31304/2144286977.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data[f'emb_{i}'] = test_data['embeddings'].apply(lambda x: x[i])
/tmp/ipykernel_31304/2144286977.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [4]:
print(data.head())
print(test_data.head())

   LEN       AWL       TTR   ASL  AVPS  ASL.AVPS       mtld  \
0   10  5.000000  0.900000  10.0   3.0      30.0  12.000000   
1   15  4.400000  1.000000  15.0   4.0      60.0  15.000000   
2   12  4.166667  0.916667  12.0   1.0      12.0  10.000000   
3   10  4.700000  1.000000  10.0   0.0       0.0  33.880000   
4   78  4.987179  0.756410  78.0   8.0     624.0  95.134118   

   num_subordinate_clauses   DCRS   FKG  ...  emb_1014  emb_1015  emb_1016  \
0                        0  16.76   4.8  ... -0.183455  0.211337  0.183684   
1                        0  20.12   4.0  ... -0.261934 -0.085224 -0.038385   
2                        0  18.12   3.3  ... -0.052533 -0.155520 -0.077319   
3                        0  18.12   3.3  ... -0.137623 -0.094990 -0.229675   
4                        0  19.27  30.2  ... -0.027255 -0.019407  0.218753   

   emb_1017  emb_1018  emb_1019  emb_1020  emb_1021  emb_1022  emb_1023  
0 -0.010561 -0.030421  0.294049  0.013648 -0.093964  0.140762  0.047726  
1 -0

## Prepare data for training and validation

In [3]:
# Preparing the data for training and validation
X = data.drop('difficulty_encoded', axis=1)
y = data['difficulty_encoded']

# Standardizing the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.01, random_state=None)

KeyError: "['difficulty_encoded'] not found in axis"

In [ ]:
from sklearn.model_selection import cross_val_score


svm_model = SVC(kernel='rbf')

# Number of folds for cross-validation
k = 5

# Perform K-fold cross-validation
cv_scores = cross_val_score(svm_model, X, y, cv=k)

cv_scores.mean(), cv_scores.std()

## Hyperparameter tuning, training and validation of training data

In [ ]:
# Define parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': ['scale', 'auto'],  # Kernel coefficient
}

# Create a GridSearchCV object for an SVM with RBF kernel
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=5, verbose=2, n_jobs=-1)

# Perform grid search
grid_search.fit(X_scaled, y)

# Print best parameters
print("Best parameters found: ", grid_search.best_params_)

# Use the best estimator for validation
best_svm = grid_search.best_estimator_

# Classification Metrics on Validation Set
#val_predictions = best_svm.predict(X_val)
#print("Classification Report on Validation Set:")
#print(classification_report(y_val, val_predictions))

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

### Predictions on test dataset

In [ ]:
# Prepare test data (for which we don't have labels)
X_test = test_data
X_test_scaled = scaler.transform(X_test)  # Use the same scaler as for the training data

# Make predictions on test data
test_predictions = best_svm.predict(X_test_scaled)

# Display predictions for test data
for idx, prediction in enumerate(test_predictions):
    print(f"Test Data ID {idx}: Predicted Difficulty: {prediction}")


In [ ]:
cefr_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}

# Re-read the test_data to get the 'id' column back
test_data = pd.read_csv('/home/nathan/OneDrive/GitHub/Nvidia/data/Cleaned_Enhanced_test.csv')

# Apply the mapping to your predictions
test_data['difficulty'] = test_predictions
test_data['difficulty'] = test_data['difficulty'].map(cefr_mapping)

# Save the 'id' and 'CEFR_difficulty' columns to a new CSV file
test_data[['id', 'difficulty']].to_csv('Data/Nvidia_submission.csv', index=False)

print("Predictions saved to Nvidia_submission.csv")

In [ ]:
print(data['embeddings'].head())

## K-cross validation

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from ast import literal_eval

data = pd.read_csv('/home/nathan/OneDrive/GitHub/Nvidia/data/Cleaned_Enhanced_training.csv')
# Convert the 'embeddings' from string representation to numerical format
data['embeddings'] = data['embeddings'].apply(literal_eval)

# Checking the first entry to ensure correct conversion
print("Sample Embedding (Post Conversion):", data['embeddings'].iloc[0])

X = data[['embeddings', 'LEN', 'DCRS', 'FKG', 'TTR', 'ARI', 'CLI', 'ASL']] # Update this line with your feature names
X = X.apply(lambda x: np.hstack(x), axis=1).tolist() # Combine embeddings with new features
X = np.array(X)

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
from sklearn.preprocessing import PolynomialFeatures

y = data['difficulty_encoded']

# Define the SVM model with RBF kernel
svm_model = SVC(kernel='rbf')

# Number of folds for cross-validation
k = 5

# Perform K-fold cross-validation
cv_scores = cross_val_score(svm_model, X_scaled, y, cv=k)

cv_scores.mean(), cv_scores.std()

In [ ]:

from sklearn.model_selection import GridSearchCV

# Define parameter range for grid search
param_grid = {
    'C': [0.1, 1, 10, 100],  # Example values, adjust as needed
    'gamma': [0.001, 0.01, 0.1, 1]  # Example values, adjust as needed
}

# Create a grid search object
grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=k)

# Perform grid search
grid_search.fit(X_scaled, y)

# Best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)


In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2)  # You can adjust the degree
X_poly = poly.fit_transform(X_scaled)